In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import yaml
from primaite.simulator.sim_container import Simulation
from primaite.simulator.network.hardware.nodes.computer import Computer
from primaite.simulator.network.hardware.nodes.server import Server
from primaite.simulator.network.hardware.nodes.switch import Switch
from primaite.simulator.network.hardware.nodes.router import Router

from primaite.simulator.system.applications.database_client import DatabaseClient
from primaite.simulator.system.services.database_service import DatabaseService
from primaite.simulator.system.services.dns_client import DNSClient
from primaite.simulator.system.services.dns_server import DNSServer
from primaite.simulator.system.services.red_services.data_manipulation_bot import DataManipulationBot


from primaite.simulator.network.hardware.nodes.router import ACLAction
from primaite.simulator.network.transmission.network_layer import IPProtocol
from primaite.simulator.network.transmission.transport_layer import Port



In [5]:
# import yaml


class PrimaiteSession:

    def __init__(self):
        self.simulation: Simulation
        self.agents = []

    @classmethod
    def from_config(cls, cfg_path):
        ref_to_uuid = {}

        game = cls()
        with open(cfg_path, 'r') as file:
            conf = yaml.safe_load(file)
        
        #1. create nodes 
        sim = Simulation()
        net = sim.network
        nodes_cfg = conf['simulation']['network']['nodes']
        links_cfg = conf['simulation']['network']['links']
        for node_cfg in nodes_cfg:
            node_ref = node_cfg['ref']
            n_type = node_cfg['type']
            if n_type == 'computer':
                new_node = Computer(hostname = node_cfg['hostname'], 
                                    ip_address = node_cfg['ip_address'], 
                                    subnet_mask = node_cfg['subnet_mask'], 
                                    default_gateway = node_cfg['default_gateway'],
                                    dns_server = node_cfg['dns_server'])
            elif n_type == 'server':
                new_node = Server(hostname = node_cfg['hostname'], 
                                    ip_address = node_cfg['ip_address'], 
                                    subnet_mask = node_cfg['subnet_mask'], 
                                    default_gateway = node_cfg['default_gateway'],
                                    dns_server = node_cfg.get('dns_server'))
            elif n_type == 'switch':
                new_node = Switch(hostname = node_cfg['hostname'],
                                  num_ports = node_cfg.get('num_ports'))
            elif n_type == 'router':
                new_node = Router(hostname=node_cfg['hostname'],
                                  num_ports = node_cfg.get('num_ports'))
                if 'ports' in node_cfg:
                    for port_num, port_cfg in node_cfg['ports'].items():
                        new_node.configure_port(port=port_num, 
                                                ip_address=port_cfg['ip_address'],
                                                subnet_mask=port_cfg['subnet_mask'])
                if 'acl' in node_cfg:
                    for r_num, r_cfg in node_cfg['acl'].items():
                        # excuse the uncommon walrus operator ` := `. It's just here as a shorthand, so that we can do
                        # both of these things once: check if a key is defined, access and convert it to a 
                        # Port/IPProtocol. TODO Refactor
                        new_node.acl.add_rule(
                            action = ACLAction[r_cfg['action']],
                            src_port = None if not (p:=r_cfg.get('src_port')) else Port[p],
                            dst_port = None if not (p:=r_cfg.get('dst_port')) else Port[p],
                            protocol = None if not (p:=r_cfg.get('protocol')) else IPProtocol[p],
                            src_ip_address = r_cfg.get('ip_address'),
                            dst_ip_address = r_cfg.get('ip_address'),
                            position = r_num
                        )
            if 'services' in node_cfg:
                for service_cfg in node_cfg['services']:
                    service_ref = service_cfg['ref']
                    service_type = service_cfg['type']
                    service_types_mapping = {
                        'DNSClient': DNSClient, # key is equal to the 'name' attr of the service class itself.
                        'DNSServer' : DNSServer,
                        'DatabaseClient': DatabaseClient,
                        'DatabaseService': DatabaseService,
                        # 'database_backup': ,
                        'DataManipulationBot': DataManipulationBot,
                        # 'web_browser'
                    }
                    if service_type in service_types_mapping:
                        new_node.software_manager.install(service_types_mapping[service_type])
                        service_obj = new_node.software_manager.software[service_type]
                        ref_to_uuid[service_ref] = service_obj.uuid
                    else:
                        print(f"service type not found {service_type}")


            net.add_node(new_node)
            ref_to_uuid[node_ref] = new_node.uuid


        #2. start/setup simulation objects
        #3. create agents
        #4. set up agents' actions and observation spaces.
        game.simulation = sim
        return game

s = PrimaiteSession.from_config('example_config.yaml')
# print(s.simulation.describe_state())

2023-09-26 10:51:10,388: Added node 48e6cb0b-f351-47f6-b837-df9443f9db26 to Network 7250d818-ec1b-4940-bb87-8e05fea87fe9
2023-09-26 10:51:10,390::WARNING::primaite.simulator.network.container::181::Can't add node 48e6cb0b-f351-47f6-b837-df9443f9db26. It is already in the network.
2023-09-26 10:51:10,394: Added node 6a969d4d-e0af-402e-b576-2a787505f7c7 to Network 7250d818-ec1b-4940-bb87-8e05fea87fe9
2023-09-26 10:51:10,397: Added node c58e6f17-dbf1-4c6a-9dbf-d60883c6d948 to Network 7250d818-ec1b-4940-bb87-8e05fea87fe9
2023-09-26 10:51:10,401: Added node 7f2a418d-2d0b-4f02-beb3-5703fc5035c8 to Network 7250d818-ec1b-4940-bb87-8e05fea87fe9
2023-09-26 10:51:10,408: Added node 967417fa-2300-4ee1-8ba0-7a4d055d5d30 to Network 7250d818-ec1b-4940-bb87-8e05fea87fe9
2023-09-26 10:51:10,413: Added node 80c1c99b-4c7a-41fb-86f0-b93c35c3b497 to Network 7250d818-ec1b-4940-bb87-8e05fea87fe9
2023-09-26 10:51:10,418: Added node 9a11dd40-9243-4510-9b43-9f247f669ad2 to Network 7250d818-ec1b-4940-bb87-8e05fe

service type not found DatabaseBackup
service type not found WebBrowser


In [ ]:
print(s.simulation.describe_state())